# Lecture 4: Homework

Today we gonna learn how to choose between ML models, based on data type. Your task would be to predict the edibility of a mushroom based on sample descriptions (binary classification problem)
The tricky part here is that 95% of the features are of categorical type. 
That's the one where we would (usually) prefer tree-based algorithms over linear methods
Although this dataset was originally contributed to the UCI Machine Learning repository nearly 30 years ago, mushroom hunting (otherwise known as "shrooming") is enjoying new peaks in popularity. Learn which features spell certain death and which are most palatable in this dataset of mushroom characteristics. And how certain can your model be?
This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom drawn from The Audubon Society Field Guide to North American Mushrooms (1981). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like "leaflets three, let it be'' for Poisonous Oak and Ivy.
More information can be found here
Please find below correspondent google form to submit your answers

In [38]:
# library import
import pandas as pd
import numpy as np
from os.path import join as pjoin
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

# preprocessing / validation
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
)
# ML models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# metrics
from sklearn.metrics import classification_report, f1_score

In [39]:
# read data
DATA_DIR = 'E:\\PycharmProjects\\Machine_L\\Data_Science_Club\\third_'
df_train = pd.read_csv(pjoin(DATA_DIR, '4-mushrooms-train.csv'), engine='c')
df_test = pd.read_csv(pjoin(DATA_DIR, '4-mushrooms-test.csv'), engine='c')
print(df_train.shape, df_test.shape)

(6499, 23) (1625, 22)


In [40]:
# let's see what data looks like
df_train.head()

,target,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,stalk_shape,stalk_root,stalk_surface_above_ring,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,0,convex,scaly,brown,bruises,pungent,free,close,narrow,white,enlarging,equal,smooth,smooth,white,white,partial,white,one,pendant,brown,scattered,urban
1,1,flat,fibrous,gray,bruises,none,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,white,partial,white,one,pendant,brown,several,woods
2,0,flat,smooth,brown,no,none,attached,close,broad,orange,enlarging,missing,smooth,smooth,orange,orange,partial,orange,one,pendant,brown,several,leaves
3,1,convex,fibrous,gray,bruises,none,free,close,broad,brown,tapering,bulbous,smooth,smooth,white,white,partial,white,one,pendant,black,solitary,woods
4,0,knobbed,smooth,brown,no,foul,free,close,narrow,buff,tapering,missing,silky,smooth,pink,pink,partial,white,one,evanescent,white,several,paths


In [41]:
# for convenient calculations, let us merge train with test
df = pd.concat([df_train, df_test], axis=0)
# add column for filtering train/test
df['is_train'] = True
df.loc[df.target.isnull(), 'is_train'] = False
# check shapes
print(df.shape)
# check labels
df.is_train.value_counts()

(8124, 24)


True     6499
False    1625
Name: is_train, dtype: int64

# Task 1. Which feature has the highest amount of unique values? (joint dataset)

In [42]:
# your code goes here
# ---------------------------------------------------------------
most_diversive = df.columns[pd.Series([len(df[elem].value_counts()) for elem in df.columns]).idxmax()]
# ---------------------------------------------------------------

print(most_diversive)

gill_color


# Task 2
As a preparation, one would spend up to 15-30 minutes on exploratory data analysis (EDA) - make sure you understand how features are distributed in train/test, what they look like, are they ordinal/binary/categorical before moving further 
While doing it, please answer the questions

2.1 Are there any features, obviously redundant to train on? If yes - what are they and why it's better to remove them?


In [43]:
print(df.shape)
# your code/hardcoded list goes here
# ---------------------------------------------------------------
redundant_columns = ['veil_type']
# ---------------------------------------------------------------
# lets drop these columns from joint dataset
df = df.drop(redundant_columns, axis=1, errors='ignore')
print(df.shape)

(8124, 24)
(8124, 23)


2.2 How many features (excluding target variable and train/test indexing columns) are:
categorical (more than 2 unique values, no explicit ordering)
ordinal (more than 2 unique values, explicit ordering)
binary (2 unique values, doesn't matter whether it has ordering or is "yes/no" styled) 

In [44]:
# your code goes here
# ---------------------------------------------------------------
binary_cols = sorted(['bruises', 'gill_attachment', 'gill_size', 'gill_spacing', 'stalk_shape'])
ordinal_cols = ['ring_number']
categorical_cols = sorted(['cap_color',
 'cap_shape',
 'cap_surface',
 'gill_color',
 'habitat',
 'odor',
 'population',
 'ring_type',
 'spore_print_color',
 'stalk_color_above_ring',
 'stalk_color_below_ring',
 'stalk_root',
 'stalk_surface_above_ring',
 'stalk_surface_below_ring',
 'veil_color',])

# ---------------------------------------------------------------
print('categorical: {}\nordinal: {}\nbinary: {}'.format(
    len(categorical_cols), len(ordinal_cols), len(binary_cols)))

categorical: 15
ordinal: 1
binary: 5


In [46]:
# To be used in training, data must be properly encoded
from collections import defaultdict

# function to encode categorical data


def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))


# label encode data (categorical + binary)
__encode_categorical(df_list=[df], cat_cols=categorical_cols+binary_cols)
# make sure you encode the only ordinal column in correct order
df[ordinal_cols[0]] = df[ordinal_cols[0]].map({'none': 0, 'one': 1, 'two': 2})

# define useful feature columns to be used for training
# (union of all columns discussed above)
columns_to_use = ordinal_cols + binary_cols + categorical_cols

# Task 3. Prepare cross-validation strategy and perform comparison of 2 baseline models (linear vs tree-based)

Briefly about Validation / Cross-Validation
Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called overfitting. 
To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set X_test, y_test. 
Note that the word “experiment” is not intended to denote academic use only, because even in commercial settings machine learning usually starts out experimentally.
When evaluating different settings (“hyperparameters”) for estimators, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. 
This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. 
To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.
However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.
A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:
A model is trained using k-1 of the folds as training data;
the resulting model is validated on the remaining part of the data 
(i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. 
This approach can be computationally expensive, but does not waste too much data (as it is the case when fixing an arbitrary test set), which is a major advantage in problem such as inverse inference where the number of samples is very small.
Some classification problems can exhibit a large imbalance in the distribution of the target classes: for instance there could be several times more negative samples than positive samples. 
In such cases it is recommended to use stratified sampling as implemented in sklearn's StratifiedKFold and StratifiedShuffleSplit to ensure that relative class frequencies is approximately preserved in each train and validation fold.
More details about different cross-validation strategies, implemented in sklearn, can be found here

In [6]:
from os import cpu_count

n_jobs = max(cpu_count()-1, 1)
# your code goes here
# ---------------------------------------------------------------
# cross-validation iterator
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
# xtrain, ytrain, DataFrame-like
xtrain = df.loc[df.is_train, columns_to_use]
ytrain = df[df.is_train].loc[:, 'target']
# ---------------------------------------------------------------

# create Decision Tree with default params, max_depth=3, random_state=42
dt = DecisionTreeClassifier(max_depth = 3, random_state = 42)
# estimate its f1-score with cross-validation (cross_val_score)
# your code goes here
# ---------------------------------------------------------------
scores_dt = cross_val_score(estimator=dt, X=xtrain, y=ytrain, scoring='f1', cv=kf, n_jobs=-1).mean()
print('DT scoring: {:.2f}'.format(scores_dt))
# ---------------------------------------------------------------
# create Logistic Regression with default params, random_state=42
lr = LogisticRegression(random_state = 42)

# estimate its f1-score with cross-validation
# your code goes here
# ---------------------------------------------------------------
scores_lr = cross_val_score(estimator=lr, X=xtrain, y=ytrain, scoring='f1', cv=kf, n_jobs=-1).mean()
print('LR scoring: {:.2f}'.format(scores_lr))

DT scoring: 0.90
LR scoring: 0.89



1. Why is a score of Linear Regression lower than correspondent one of DT?
2. Is everything OK with the data format for linear models? (revision of 2 previous lectures). 
3. If not, what else you should do to use the data appropriately for linear models?
Why didn't point 1. affect Decision Tree performance?


1. Линейная модель показала результат, хуже чем дерево, потому что деревья восстанавливают более сложные закономерности и как следствие могут дают более сложные модели, однако это приводит к переобучению.
2. Данные имеют множество категориальных переменных, с которыми линейные модели работают не очень хорошо.
3. Алгоритмы на основе деревьев принятия решений хороши тем, что могут обрабатывать различные типы признаков, не требуют масштабирования данных, в отличии от линейных моделей.

# Task 4. Now it's time to do some hyperparam tuning
Perform suitable hyperparam tuning using created above cross-validation strategy 
Main parameters to perform grid-search for:
max_depth (1,2,...None)
min_samples_leaf (1,2,...)
criterion (gini, entropy)
weight (none, balanced)
max_features (sqrt(features), 50%, 75%, all of them, ...)
other params available, see documentation
So - use your fantasy for filling-in abovementioned lists
You should receive a gain of 0.01 in f1-score or higher 
(current benchmark = +0.0268 gain)

In [8]:
%%time
# your code goes here
# ---------------------------------------------------------------
# create base model (DT, random state = 42)
estimator = DecisionTreeClassifier(random_state = 42)

# create parameter grid
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
X_train = df[df['is_train'] == True].loc[:, :'veil_color']
y_train = df[df['is_train'] == True].loc[:, 'target']
# create parameter grid
max_depth=range(1,12)
max_features = range(1,X_train.shape[1])
min_samples_leaf= range(1,10)
max_leaf_nodes = range(2,10)

params = {
          'criterion':['gini', 'entropy'],
          'class_weight': [None , 'balanced'] ,
          'splitter': ['best' , 'random'],
          'presort': [True , False],
          'max_depth': [ 4,5,6,7 ,8, 10 , 15 , 25 , None], 
          'min_samples_leaf': [ 1, 2, 3, 4, 5, 6 , 8 , 10 , 15],
          'min_samples_split': [2 , 3 , 5 , 7 , 9 , 12 , 15],
          'max_leaf_nodes': [10 , 20 , 35 ,  50 , 80 , 100 , None] ,
          'max_features': [ 8, 10 ,13 , 15 ,17, 19,  21],
         }

# create grid search object (without veil_type)
gs1 = GridSearchCV(
    estimator=estimator,  # base model
    param_grid=params,  # params grid to search within
    cv=kf,  # cross-validation strategy
    error_score=1,  # warnings only
    scoring='f1',  # f1-score
    # thread count, the higher count - the faster
    n_jobs=-1,
    verbose=2,  # messages about performed actions
)

# perform grid search on TRAIN dataset ('is_train' filtering)
gs1.fit(X=X_train, y=y_train)

Fitting 5 folds for each of 444528 candidates, totalling 2222640 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1031 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 3467 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 6863 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 11243 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 16583 tasks      | elapsed:   45.8s
[Parallel(n_jobs=-1)]: Done 22907 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 30191 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 38459 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 47687 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 57899 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 69071 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 81227 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 94343 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 108443 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 123503 tasks    

Wall time: 1h 55min 21s


In [9]:
# extract best score on cross-validation without veil_type
best_score1 = gs1.best_score_
# extract the estimator (DT) with best params on cross-validation
best_dt1 = gs1.best_estimator_
# check gain in f1-score
print('f1-score best: {:.4f}, +{:.4f} better than baseline'.format(best_score1, (best_score1 - scores_dt)))

f1-score best: 0.9283, +0.0271 better than baseline


In [10]:
best_dt1

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
            max_features=13, max_leaf_nodes=80, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=7, min_weight_fraction_leaf=0.0,
            presort=True, random_state=42, splitter='random')

In [36]:
# check performance on holdout dataset, unseen before (filter 'is_train' == False)

# your code goes here
# ---------------------------------------------------------------
# appropriate df_test data subset from 'df' dataframe
xtest = df[df['is_train'] == False].loc[:, :'veil_color']
#xtest1 = df[df['is_train'] == False].loc[:, :'veil_color']# ...
# fit baseline model 'dt' on xtrain, ytrain (because it's not fitted yet)
dt.fit(xtrain, ytrain)
# ---------------------------------------------------------------
DATA_DIR = 'data'
# baseline model
y_true = pd.read_csv('E:\\PycharmProjects\\Machine_L\\Data_Science_Club\\third_\\4-mushrooms-y_test.csv')
y_pred_baseline = dt.predict(xtest)

print('Base on train:   {:.4f}\nBase on holdout: {:.4f}\ndiff: {:.4f}'.format(scores_dt, f1_score(y_true, y_pred_baseline),
    scores_dt - f1_score(y_true, y_pred_baseline)))

# best model
y_pred_best = best_dt1.predict(xtest)

print('\nBest on train:   {:.4f}\nBest on holdout: {:.4f}\ndiff: {:.4f}'.format(
    best_score1, 
    f1_score(y_true, y_pred_best),
    best_score1 - f1_score(y_true, y_pred_best)
))

Base on train:   0.9012
Base on holdout: 0.8888
diff: 0.0125

Best on train:   0.9283
Best on holdout: 0.9205
diff: 0.0078


Now you can see that 
absolute values of f1-score is higher and distance between train|holdout is lower 
for best model in comparison to baseline

Bonus question:
Consider two possibilities:
(a) you have trained one best (on cross-validation) Decision Tree
(b) you randomly choose 25 subsets of 70% of training data, fits "overfitted" (max_depth=None) Decision Trees on it - each of them performs slightly worse than Tree in (a), and then average predicted results over all 25 models (overfitted trees)
Which one of them would most likely give the best results on hold-out dataset? What makes you think that way?

Второй вариант (b) сработает гораздо лучше. В этом случае будет использоваться ансамбли деревьев (RandomForest or GradientBoosting), которые показывают гораздо лучшие результаты по сравнению с деревьями. Также ансамбли деревьев, как правило, имеют лучшую обобщающую способность и поэтому менее склонны к переобучению, нежели деревья принятия решений.